In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne
import glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData
from ipynb.fs.full.feature_selection import get_features
from ipynb.fs.full.fullDataExtraction import getLabelsAndMatfiles
from ipynb.fs.full.fullDataExtraction import compute_diffEnt

Imported 0.2.03 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



In [3]:
def getAllFeatureData():
    x=getRawArrayData()
    matfiles,label_list = getLabelsAndMatfiles(glob.glob('Preprocessed_EEG/*.mat'))
    selected_funcs = get_features()
    combined = np.zeros((1,len(selected_funcs)*62+2))
    for subs in x:
        sliced_epochs={}
        for clip in x[subs]:
            sliced_epochs[clip]= mne.make_fixed_length_epochs(x[0][clip], duration=1, preload=True,verbose=0)
        for cut in sliced_epochs:
            label_used=label_list[cut]
            epoch_array=mne.Epochs.get_data(sliced_epochs[cut])
            extracted_data=extract_features(epoch_array,200,selected_funcs)
            label_col = np.zeros((extracted_data.shape[0],1)) + label_used
            cur_sub = np.zeros((extracted_data.shape[0],1)) + (subs+1)
            oneCom = np.hstack((extracted_data,label_col))
            oneCom = np.hstack((oneCom,cur_sub))
            combined = np.vstack((combined,oneCom))
    combined = np.delete(combined, 0, axis=0)
    return combined